#  Project: Banff Traffic Management

**Group Name:** Mobility Innovators

**Team Members & Roles:**
* **Aquiles Escarra:** Project Manager
* **Inderjeet Singh:** Lead Data Scientist
* **Aditya Mehta:** Project Coordinator
* **Yungvir Singh:** Scrum Master

---

##  Executive Summary: Weather Data Preprocessing

**Objective:**
This notebook prepares the **Meteorological Data** (temperature, precipitation, snow) sourced from Environment Canada. Weather is a primary external factor influencing visitor behavior and, consequently, parking demand.

**Methodology:**
1.  **Data Extraction:** We load raw daily climate reports for 2024 and 2025 from the Banff CS station.
2.  **Data Cleaning:** We remove redundant "flag" columns and handle missing values (e.g., filling null snow-on-ground values with 0 for summer months).
3.  **Feature Engineering:** We create binary impact flags such as:
    * `has_precip`: Indicates if rain or snow fell that day.
    * `is_bad_weather`: Identifies extreme conditions (e.g., < -25°C or heavy snow) likely to deter tourists.
4.  **Standardization:** We rename columns to code-friendly snake_case (e.g., `Max Temp (°C)` -> `max_temp`) and ensure date formats match the other datasets.

**Output:**
The output is a clean, daily weather time-series CSV (`df_banff_weather.csv`) ready to be merged with hourly parking and traffic data in the Consolidation notebook.

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
# Load traffic volume datasets
df_1 = pd.read_csv('en_climate_daily_AB_3050519_2024_P1D.csv')
df_2 = pd.read_csv('en_climate_daily_AB_3050519_2025_P1D.csv')

In [ ]:
#Setting up notebook so that all columns are displayed
pd.set_option('display.max_columns', None)

In [ ]:
df_1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 366 entries, 0 to 365
Data columns (total 31 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   Longitude (x)              366 non-null    float64
 1   Latitude (y)               366 non-null    float64
 2   Station Name               366 non-null    object 
 3   Climate ID                 366 non-null    int64  
 4   Date/Time                  366 non-null    object 
 5   Year                       366 non-null    int64  
 6   Month                      366 non-null    int64  
 7   Day                        366 non-null    int64  
 8   Data Quality               0 non-null      float64
 9   Max Temp (°C)              364 non-null    float64
 10  Max Temp Flag              2 non-null      object 
 11  Min Temp (°C)              364 non-null    float64
 12  Min Temp Flag              2 non-null      object 
 13  Mean Temp (°C)             364 non-null    float64

In [ ]:
df_1.head()

,Longitude (x),Latitude (y),Station Name,Climate ID,Date/Time,Year,Month,Day,Data Quality,Max Temp (°C),Max Temp Flag,Min Temp (°C),Min Temp Flag,Mean Temp (°C),Mean Temp Flag,Heat Deg Days (°C),Heat Deg Days Flag,Cool Deg Days (°C),Cool Deg Days Flag,Total Rain (mm),Total Rain Flag,Total Snow (cm),Total Snow Flag,Total Precip (mm),Total Precip Flag,Snow on Grnd (cm),Snow on Grnd Flag,Dir of Max Gust (10s deg),Dir of Max Gust Flag,Spd of Max Gust (km/h),Spd of Max Gust Flag
0,-115.55,51.19,BANFF CS,3050519,2024-01-01,2024,1,1,NaN,2.3,NaN,-5.1,NaN,-1.4,NaN,19.4,NaN,0.0,NaN,NaN,NaN,NaN,NaN,0.0,NaN,12.0,NaN,22.0,NaN,32.0,NaN
1,-115.55,51.19,BANFF CS,3050519,2024-01-02,2024,1,2,NaN,-2.1,NaN,-8.5,NaN,-5.3,NaN,23.3,NaN,0.0,NaN,NaN,NaN,NaN,NaN,0.0,NaN,12.0,NaN,22.0,NaN,32.0,NaN
2,-115.55,51.19,BANFF CS,3050519,2024-01-03,2024,1,3,NaN,-2.8,NaN,-8.7,NaN,-5.8,NaN,23.8,NaN,0.0,NaN,NaN,NaN,NaN,NaN,0.0,NaN,12.0,NaN,24.0,NaN,32.0,NaN
3,-115.55,51.19,BANFF CS,3050519,2024-01-04,2024,1,4,NaN,0.6,NaN,-6.2,NaN,-2.8,NaN,20.8,NaN,0.0,NaN,NaN,NaN,NaN,NaN,0.0,NaN,12.0,NaN,22.0,NaN,31.0,NaN
4,-115.55,51.19,BANFF CS,3050519,2024-01-05,2024,1,5,NaN,-0.5,NaN,-7.0,NaN,-3.7,NaN,21.7,NaN,0.0,NaN,NaN,NaN,NaN,NaN,0.0,NaN,12.0,NaN,NaN,NaN,NaN,NaN


In [ ]:
def clean_weather_data(df):
    # 1. Select useful columns (removing flags and empty fields)
    # Note: We do NOT include 'Total Rain' or 'Total Snow' because they were confirmed empty.
    cols_to_keep = [
        'Date/Time',
        'Max Temp (°C)',
        'Min Temp (°C)',
        'Mean Temp (°C)',
        'Total Precip (mm)',
        'Snow on Grnd (cm)'
    ]

    # Keep only the columns that exist in the dataset
    existing_cols = [c for c in cols_to_keep if c in df.columns]
    df_clean = df[existing_cols].copy()

    # 2. Rename columns to code-friendly snake_case format
    rename_map = {
        'Date/Time': 'date',
        'Max Temp (°C)': 'max_temp',
        'Min Temp (°C)': 'min_temp',
        'Mean Temp (°C)': 'mean_temp',
        'Total Precip (mm)': 'total_precip',
        'Snow on Grnd (cm)': 'snow_on_ground'
    }
    df_clean = df_clean.rename(columns=rename_map)

    # 3. Convert date column to datetime
    df_clean['date'] = pd.to_datetime(df_clean['date'])

    # 4. Handle Missing Values (Critical for ML models)

    # Snow on ground: If NaN, assume 0 (typical for summer)
    if 'snow_on_ground' in df_clean.columns:
        df_clean['snow_on_ground'] = df_clean['snow_on_ground'].fillna(0)

    # Precipitation: If NaN, assume 0
    if 'total_precip' in df_clean.columns:
        df_clean['total_precip'] = df_clean['total_precip'].fillna(0)

    # Temperatures: If missing (those 2 days we found), forward-fill using previous day
    temp_cols = ['max_temp', 'min_temp', 'mean_temp']
    for col in temp_cols:
        if col in df_clean.columns:
            df_clean[col] = df_clean[col].ffill()

    return df_clean


# --- EXECUTION ---

# 1. Clean df_1 (2024)
df_2024_clean = clean_weather_data(df_1)

# 2. Assume df_2 is your 2025 raw dataset.
# If you haven't loaded it yet, use: df_2 = pd.read_csv("weather_2025.csv")
# Then:
# df_2025_clean = clean_weather_data(df_2)

# 3. Concatenate 2024 and 2025 vertically
# If you only have df_1 ready, just use that. Otherwise:
# weather_final = pd.concat([df_2024_clean, df_2025_clean], ignore_index=True)

# Showing final 2024 clean structure as example
print("Final cleaned structure (2024):")
print(df_2024_clean.info())
print(df_2024_clean.head())


Final cleaned structure (2024):
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 366 entries, 0 to 365
Data columns (total 6 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   date            366 non-null    datetime64[ns]
 1   max_temp        366 non-null    float64       
 2   min_temp        366 non-null    float64       
 3   mean_temp       366 non-null    float64       
 4   total_precip    366 non-null    float64       
 5   snow_on_ground  366 non-null    float64       
dtypes: datetime64[ns](1), float64(5)
memory usage: 17.3 KB
None
        date  max_temp  min_temp  mean_temp  total_precip  snow_on_ground
0 2024-01-01       2.3      -5.1       -1.4           0.0            12.0
1 2024-01-02      -2.1      -8.5       -5.3           0.0            12.0
2 2024-01-03      -2.8      -8.7       -5.8           0.0            12.0
3 2024-01-04       0.6      -6.2       -2.8           0.0            12.0
4 2024-01-05 

In [ ]:
# 1. Clean the 2025 dataset (using the same logic previously validated for 2024)
# Make sure df_2 is your raw 2025 weather dataset
df_2025_clean = clean_weather_data(df_2)

# 2. Concatenate both years vertically
weather_final = pd.concat([df_2024_clean, df_2025_clean], ignore_index=True)

# Sort by date to ensure proper temporal order
weather_final = weather_final.sort_values('date')

print(f"✅ Complete Weather Dataset (2024–2025): {weather_final.shape[0]} rows.")

✅ Complete Weather Dataset (2024–2025): 731 rows.


In [ ]:
weather_final.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 731 entries, 0 to 730
Data columns (total 6 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   date            731 non-null    datetime64[ns]
 1   max_temp        731 non-null    float64       
 2   min_temp        731 non-null    float64       
 3   mean_temp       731 non-null    float64       
 4   total_precip    731 non-null    float64       
 5   snow_on_ground  731 non-null    float64       
dtypes: datetime64[ns](1), float64(5)
memory usage: 34.4 KB


In [ ]:
weather_final.head()

,date,max_temp,min_temp,mean_temp,total_precip,snow_on_ground
0,2024-01-01,2.3,-5.1,-1.4,0.0,12.0
1,2024-01-02,-2.1,-8.5,-5.3,0.0,12.0
2,2024-01-03,-2.8,-8.7,-5.8,0.0,12.0
3,2024-01-04,0.6,-6.2,-2.8,0.0,12.0
4,2024-01-05,-0.5,-7.0,-3.7,0.0,12.0


In [ ]:
# ---------------------------------------------------------
# APPLY WEATHER LOGIC TO 'weather_final'
# ---------------------------------------------------------

# Define Thresholds
THRESH_PRECIP = 10.0   # >10mm (Heavy rain or snowstorm)
THRESH_COLD = -25.0    # < -25°C (Extreme cold)

# 1. has_precip: Is the ground wet/slippery?
# Impact: Slower traffic, people tend to look for parking closer.
weather_final['has_precip'] = (weather_final['total_precip'] > 0).astype(int)

# 2. is_bad_weather: Extreme weather conditions
# Impact: Strong reduction of visitors to the town.
weather_final['is_bad_weather'] = np.where(
    (weather_final['total_precip'] >= THRESH_PRECIP) |
    (weather_final['min_temp'] <= THRESH_COLD),
    1, 0
)

# ---------------------------------------------------------
# VALIDATION
# ---------------------------------------------------------
extreme_days = weather_final['is_bad_weather'].sum()
total_days = len(weather_final)

print(f"✅ Columns added successfully.")
print(f"   - Days with precipitation (has_precip): {weather_final['has_precip'].sum()}")
print(f"   - Extreme Weather Days (is_bad_weather): {extreme_days} out of {total_days} ({round(extreme_days/total_days*100, 1)}%)")


✅ Columns added successfully.
   - Days with precipitation (has_precip): 230
   - Extreme Weather Days (is_bad_weather): 36 out of 731 (4.9%)


In [ ]:
print("\nExamples of extreme weather days (Visual check):")
print(weather_final[weather_final['is_bad_weather'] == 1][['date', 'total_precip', 'min_temp']].head(20))


Examples of extreme weather days (Visual check):
          date  total_precip  min_temp
10  2024-01-11           0.0     -32.2
11  2024-01-12           0.0     -39.8
12  2024-01-13           0.0     -40.2
13  2024-01-14           0.0     -38.3
14  2024-01-15           0.0     -36.8
17  2024-01-18           0.0     -30.7
18  2024-01-19           0.0     -28.6
59  2024-02-29          13.1      -7.2
62  2024-03-03           0.0     -25.1
120 2024-04-30          10.8      -1.3
168 2024-06-17          12.1       0.4
178 2024-06-27          17.7       5.0
179 2024-06-28          10.0       5.3
228 2024-08-16          17.8      12.2
240 2024-08-28          20.6       2.8
294 2024-10-21          14.0      -3.2
385 2025-01-20           0.0     -25.1
398 2025-02-02           0.8     -25.8
399 2025-02-03           0.1     -26.8
400 2025-02-04           0.3     -26.4


In [ ]:
weather_final.head()

,date,max_temp,min_temp,mean_temp,total_precip,snow_on_ground,has_precip,is_bad_weather
0,2024-01-01,2.3,-5.1,-1.4,0.0,12.0,0,0
1,2024-01-02,-2.1,-8.5,-5.3,0.0,12.0,0,0
2,2024-01-03,-2.8,-8.7,-5.8,0.0,12.0,0,0
3,2024-01-04,0.6,-6.2,-2.8,0.0,12.0,0,0
4,2024-01-05,-0.5,-7.0,-3.7,0.0,12.0,0,0


Save Weather data as 'df_banff_weather.csv'

In [ ]:
# Define file name and path
file_name = "df_banff_weather.csv"

# Save the DataFrame to a CSV file
weather_final.to_csv(file_name, index=False, encoding='utf-8')

# Confirm file creation
print(f"✅ DataFrame successfully saved as '{file_name}'")
print(f"Rows: {weather_final.shape[0]:,} | Columns: {weather_final.shape[1]}")

# Download the file directly to your computer
from google.colab import files
files.download(file_name)

✅ DataFrame successfully saved as 'df_banff_weather.csv'
Rows: 731 | Columns: 8


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>